## Fine tuning

In [ ]:
!pip install evaluate && pip install -U transformers

In [ ]:
import pandas as pd
from datasets import Dataset
import numpy as np
import evaluate

from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer

In [ ]:
df = pd.read_csv("./data/df_final.csv")

In [ ]:
# Asegúrate de tener solo las columnas necesarias
hf_dataset = Dataset.from_pandas(df[['review', 'sentiment']])  # o 'score' si es lo que usas

In [ ]:
label_enc = LabelEncoder()
df["label"] = label_enc.fit_transform(df["sentiment"])  # crea columna 'label' numérica

# Dataset para Hugging Face con la nueva columna
hf_dataset = Dataset.from_pandas(df[['review', 'label']])

In [ ]:
hf_dataset = hf_dataset.train_test_split(test_size=0.2, seed=42)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

def tokenize_function(example):
    return tokenizer(example["review"], padding="max_length", truncation=True)

tokenized_datasets = hf_dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/6959 [00:00<?, ? examples/s]

Map:   0%|          | 0/1740 [00:00<?, ? examples/s]

In [ ]:
# Ajusta num_labels según tus clases (por ejemplo, 2 para positivo/negativo)
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=len(set(df["label"])))

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-42-f68c0d1200e9>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jopebrui (jopebrui-universitat-de-val-ncia) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.856900,0.777367,0.690805
2,0.646300,0.740449,0.714368


Epoch,Training Loss,Validation Loss,Accuracy
1,0.856900,0.777367,0.690805
2,0.646300,0.740449,0.714368
3,0.484500,0.791617,0.700575


TrainOutput(global_step=2610, training_loss=0.662545261894606, metrics={'train_runtime': 2130.7371, 'train_samples_per_second': 9.798, 'train_steps_per_second': 1.225, 'total_flos': 5493018821907456.0, 'train_loss': 0.662545261894606, 'epoch': 3.0})

In [ ]:
trainer.save_model("sentiment-finetuned-movie-reviews")


In [ ]:
# Realiza predicciones en el conjunto de test
predictions = trainer.predict(tokenized_datasets["test"])

In [ ]:
y_pred = np.argmax(predictions.predictions, axis=1)
y_true = predictions.label_ids


In [ ]:
# Si usaste LabelEncoder antes para codificar etiquetas:
print(classification_report(y_true, y_pred, target_names=le.classes_))


              precision    recall  f1-score   support

         NEG       0.75      0.67      0.71       532
         NEU       0.47      0.58      0.52       428
         POS       0.83      0.78      0.81       780

    accuracy                           0.70      1740
   macro avg       0.68      0.68      0.68      1740
weighted avg       0.72      0.70      0.71      1740

